<a href="https://colab.research.google.com/github/martykihn/xmas_movies_analysis/blob/main/LDA_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# this code is just for Google Colab notebook word wrapping format

from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/Colab Notebooks/TextFiles

/content/drive/MyDrive/Colab Notebooks/TextFiles


In [4]:
ls

cached_lm_GPT2Tokenizer_126_text_as_list_test.csv
cached_lm_GPT2Tokenizer_126_text_as_list_test.csv.lock
cached_lm_GPT2Tokenizer_126_text_as_list_train.csv
cached_lm_GPT2Tokenizer_126_text_as_list_train.csv.lock
cached_lm_GPT2Tokenizer_126_text_nopunc.txt
cached_lm_GPT2Tokenizer_126_text_nopunc.txt.lock
data/
plots-finalformat.csv
plots_test.txt
plots_train.txt
plots_valid.txt
runs/
story_generator_checkpoint/
text_as_list.csv
text_as_list_test.csv
text_as_list_test_TOKENS.txt
text_as_list_train.csv
text_as_list_train_TOKENS.txt
text_as_string_test.csv
text_as_string_train.csv
text_newlines.txt
text_nopunc.txt
text.txt
transformers/


In [5]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer

lemtzer = WordNetLemmatizer()

def lemmatize_stemming(text):
    return lemtzer.lemmatize(text, pos='v')

# Write a function to perform the pre processing steps on the entire dataset
def preprocess(text):
    result=[]
    for token in simple_preprocess(text) :
        if token not in STOPWORDS:
            result.append(lemmatize_stemming(token))
            
    return result

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
doc_sample = 'This disk has failed many times. I would like to get it replaced.'

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['This', 'disk', 'has', 'failed', 'many', 'times.', 'I', 'would', 'like', 'to', 'get', 'it', 'replaced.']


Tokenized and lemmatized document: 
['disk', 'fail', 'time', 'like', 'replace']


In [8]:
# plots = "/content/drive/MyDrive/Colab Notebooks/TextFiles/text_newlines.txt"

In [9]:
plots = []
with open('/content/drive/MyDrive/Colab Notebooks/TextFiles/text_newlines.txt') as myfile:
  for plot in myfile:
    plot = plot.replace('\n', '')
    if plot == '':
      continue
    else:
      plots.append(str(plot))
myfile.close()

In [10]:
type(plots)

list

In [11]:
len(plots)

249

In [12]:
# create train and test splits
plots_train = plots[0:200]
plots_test = plots[201:]

In [13]:
len(plots_train)

200

In [14]:
type(plots_train)

list

In [15]:
processed_train = []
for plot in plots_train:
  processed_train.append(preprocess(plot))

In [16]:
# each plot is now a list of words in a list of plots

In [17]:
import gensim
dictionary = gensim.corpora.Dictionary(processed_train)

In [19]:
len(dictionary.keys())

2395

In [20]:
count = 0
for k, v in dictionary.iteritems():
  print(k, v)
  count += 1
  if count > 10:
    break

0 angel
1 balance
2 career
3 daughter
4 dream
5 elderly
6 estate
7 family
8 help
9 life
10 live


In [21]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_train]

In [22]:
bow_corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1)]

In [23]:
# num_topics - number of topics from corpus
# id2word - maps word ids (ints) to words (strs) - printing
# passes - training passes

In [24]:
%%time
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=2)
lda_model.save('lda.model')

CPU times: user 610 ms, sys: 47.5 ms, total: 657 ms
Wall time: 752 ms


In [25]:
for idx, topic in lda_model.print_topics(-1):
  print('Topic: {}   Word: {}\n'.format(idx, topic))

Topic: 0   Word: 0.032*"christmas" + 0.010*"holiday" + 0.009*"town" + 0.008*"family" + 0.007*"new" + 0.007*"meet" + 0.006*"love" + 0.005*"time" + 0.005*"home" + 0.005*"find"

Topic: 1   Word: 0.023*"christmas" + 0.009*"love" + 0.008*"new" + 0.008*"holiday" + 0.007*"find" + 0.005*"help" + 0.005*"plan" + 0.005*"young" + 0.005*"town" + 0.005*"family"

Topic: 2   Word: 0.027*"christmas" + 0.011*"holiday" + 0.010*"family" + 0.008*"find" + 0.007*"help" + 0.007*"life" + 0.006*"love" + 0.004*"spirit" + 0.004*"angel" + 0.004*"town"

Topic: 3   Word: 0.025*"christmas" + 0.009*"help" + 0.007*"town" + 0.006*"meet" + 0.005*"love" + 0.005*"new" + 0.005*"home" + 0.005*"santa" + 0.005*"life" + 0.005*"jack"



In [33]:
num = 100
unseen_doc = plots_test[18]
print(unseen_doc)

Julia Crawley and Ryan Mason must unite their families during Crestridges Centennial Christmas celebration in order to save their family inns from a chain resort.


In [34]:
bow_vector = dictionary.doc2bow(preprocess(unseen_doc))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
  print("Score: {}\t  Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.537270724773407	  Topic: 0.025*"christmas" + 0.009*"help" + 0.007*"town" + 0.006*"meet" + 0.005*"love"
Score: 0.3163113594055176	  Topic: 0.032*"christmas" + 0.010*"holiday" + 0.009*"town" + 0.008*"family" + 0.007*"new"
Score: 0.1241937056183815	  Topic: 0.027*"christmas" + 0.011*"holiday" + 0.010*"family" + 0.008*"find" + 0.007*"help"
Score: 0.02222418785095215	  Topic: 0.023*"christmas" + 0.009*"love" + 0.008*"new" + 0.008*"holiday" + 0.007*"find"


In [36]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7MB 8.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=37efb361b74ffdbc808ad259d703bc69b82d70b7ff4eea06ae69e8048792a5a6
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis


In [38]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.000772 -0.075702       1        1  30.690635
3      0.074465  0.025747       2        1  26.279007
1     -0.029171  0.026183       3        1  23.927521
2     -0.046066  0.023771       4        1  19.102837, topic_info=         Term       Freq      Total Category  logprob  loglift
576      find  37.000000  37.000000  Default  30.0000  30.0000
472      jack  15.000000  15.000000  Default  29.0000  29.0000
148      come  18.000000  18.000000  Default  28.0000  28.0000
1226  michael   4.000000   4.000000  Default  27.0000  27.0000
7      family  48.000000  48.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
638     small   4.024492  17.950841   Topic4  -5.8331   0.1601
337      fall   4.226391  22.667642   Topic4  -5.7842  -0.0243
735   romance   4.117536  21.326553   Topic4  -5.8103   0.0106
201      lose   3.752224  14.359740   Topic4  -5.9032   0.3133
254       new   4.345402  44.126621   Topic4  -5.7564  -0.6626

[304 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
460       2  0.681614    accept
460       4  0.227205    accept
1835      3  0.668525  adelaide
2133      1  0.950065      alec
1999      4  0.946457      andy
...     ...       ...       ...
104       2  0.267093     young
104       3  0.400639     young
104       4  0.133546     young
1949      2  0.897725       zoe
2146      1  0.950038      zoey

[478 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3])

In [39]:
from gensim import models
import warnings
warnings.filterwarnings('ignore')

In [40]:
tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

In [42]:
%%time
lda_model_tfidf = gensim.models.LdaMulticore(tfidf_corpus, num_topics=4, id2word=dictionary, passes=2)
lda_model_tfidf.save('lda_tfidf.model')

CPU times: user 574 ms, sys: 50.3 ms, total: 624 ms
Wall time: 671 ms


In [43]:
for idx, topic in lda_model_tfidf.print_topics(-1):
  print('Topic: {}  Word: {}'.format(idx, topic))

Topic: 0  Word: 0.002*"old" + 0.002*"time" + 0.002*"home" + 0.002*"new" + 0.002*"family" + 0.002*"help" + 0.002*"love" + 0.002*"dad" + 0.001*"season" + 0.001*"meet"
Topic: 1  Word: 0.002*"town" + 0.002*"nick" + 0.002*"help" + 0.002*"store" + 0.002*"santa" + 0.001*"life" + 0.001*"military" + 0.001*"angel" + 0.001*"come" + 0.001*"young"
Topic: 2  Word: 0.002*"chris" + 0.001*"discover" + 0.001*"story" + 0.001*"family" + 0.001*"carol" + 0.001*"begin" + 0.001*"plan" + 0.001*"tree" + 0.001*"town" + 0.001*"young"
Topic: 3  Word: 0.002*"find" + 0.002*"holiday" + 0.002*"life" + 0.002*"love" + 0.002*"home" + 0.001*"family" + 0.001*"new" + 0.001*"darcy" + 0.001*"fall" + 0.001*"town"


In [44]:
pyLDAvis.gensim.prepare(lda_model_tfidf, tfidf_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.019301  0.008620       1        1  28.903253
3     -0.001411 -0.020759       2        1  25.854850
2      0.014983  0.007298       3        1  23.037622
1      0.005728  0.004841       4        1  22.204275, topic_info=          Term      Freq     Total Category  logprob  loglift
997      chris  0.000000  0.000000  Default  30.0000  30.0000
121       nick  0.000000  0.000000  Default  29.0000  29.0000
1631  military  0.000000  0.000000  Default  28.0000  28.0000
696      emily  0.000000  0.000000  Default  27.0000  27.0000
1044   melanie  0.000000  0.000000  Default  26.0000  26.0000
...        ...       ...       ...      ...      ...      ...
735    romance  0.246294  1.016243   Topic4  -6.7914   0.0875
132       love  0.250913  1.300660   Topic4  -6.7728  -0.1406
113       true  0.238074  0.932238   Topic4  -6.8253   0.1399
128       turn  0.235250  0.809231   Topic4  -6.8373   0.2694
163     school  0.235373  0.937970   Topic4  -6.8367   0.1223

[264 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
576       2  0.790610  find
182       1  1.033562   old
57        1  0.883587  time, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 2])

In [47]:
# back to the plots data set - which is a list
plots[:2]

['A ski-shop owner reluctantly moves himself, his wife, and his daughter in to an estate as live-in help for an elderly widow. While struggling to balance his career and family life, he has recurring dreams about an angel.',
 'Television movie updating Charles Dickens story, "A Christmas Carol." Businesswoman Ebenita Scrooge treats her employees and customers poorly. She has no time for Christmas or the holiday spirit. On Christmas Eve, she is visited by the ghost of her dead partner Maude Marley and then by other spirits who remind her of her happy past and chronicle the bitterness and greed that have taken over her life. At last, she is shown her own death and funeral. No one is there to mourn her. This revelation shocks her into opening her heart and her checkbook.']

In [58]:
plots_list = [plot.split() for plot in plots]

In [59]:
plots_list[:2]

[['A',
  'ski-shop',
  'owner',
  'reluctantly',
  'moves',
  'himself,',
  'his',
  'wife,',
  'and',
  'his',
  'daughter',
  'in',
  'to',
  'an',
  'estate',
  'as',
  'live-in',
  'help',
  'for',
  'an',
  'elderly',
  'widow.',
  'While',
  'struggling',
  'to',
  'balance',
  'his',
  'career',
  'and',
  'family',
  'life,',
  'he',
  'has',
  'recurring',
  'dreams',
  'about',
  'an',
  'angel.'],
 ['Television',
  'movie',
  'updating',
  'Charles',
  'Dickens',
  'story,',
  '"A',
  'Christmas',
  'Carol."',
  'Businesswoman',
  'Ebenita',
  'Scrooge',
  'treats',
  'her',
  'employees',
  'and',
  'customers',
  'poorly.',
  'She',
  'has',
  'no',
  'time',
  'for',
  'Christmas',
  'or',
  'the',
  'holiday',
  'spirit.',
  'On',
  'Christmas',
  'Eve,',
  'she',
  'is',
  'visited',
  'by',
  'the',
  'ghost',
  'of',
  'her',
  'dead',
  'partner',
  'Maude',
  'Marley',
  'and',
  'then',
  'by',
  'other',
  'spirits',
  'who',
  'remind',
  'her',
  'of',
  'her',


In [61]:
from gensim.models import Word2Vec
word2vec_model = Word2Vec(plots_list, min_count=1)

In [62]:
word2vec_model

In [63]:
# my version of 'sentences' is 'plots_list'
# going to use full memory version not iterator
assert gensim.models.doc2vec.FAST_VERSION > -1

In [64]:
%%time
import multiprocessing
import os
cores = multiprocessing.cpu_count()

CPU times: user 71 µs, sys: 7 µs, total: 78 µs
Wall time: 451 µs


In [67]:
if not os.path.exists('models/word2vec.model'):
  print('start training word2vec.model')
  sentences = plots_list
  word2vec_model = gensim.models.Word2Vec(sentences, min_count=2, size=100, workers=cores)
  if not os.path.exists('models'):
    os.makedirs('models')
    word2vec_model.save('models/word2vec.model')
  else:
    word2vec_model.save('models/word2vec.model')
else:
  word2vec_model = Word2Vec.load('models/word2vec.model')

start training word2vec.model


In [69]:
word2vec_model.wv.word_vec('Christmas')

array([ 0.1970382 , -0.21689399, -0.00390408, -0.2369994 ,  0.02496545,
        0.06454246,  0.487685  ,  0.18095753,  0.01666983, -0.2974786 ,
        0.09059469,  0.15560105,  0.10792081,  0.31166247, -0.26600683,
       -0.262913  ,  0.05723181, -0.07101402, -0.11714032, -0.19389135,
        0.16362998, -0.7988008 ,  0.32564405, -0.27396655, -0.01914163,
        0.0913619 , -0.11726357, -0.48127845,  0.21724996,  0.24758317,
       -0.59108377, -0.26935256, -0.29871213, -0.07328557, -0.0928225 ,
       -0.23772073,  0.45015377, -0.21111883,  0.35377714,  0.12653787,
       -0.30799347, -0.18911698, -0.670812  ,  0.00102891, -0.19020279,
        0.12648304, -0.14020555,  0.08863872, -0.02151071,  0.37601885,
        0.12073237,  0.06452664, -0.04083474, -0.9545262 , -0.11062787,
        0.01797774,  0.0299483 , -0.57229155, -0.1631023 ,  0.6230042 ,
        0.36177853,  0.6602794 , -0.12961468, -0.08078633,  0.5503976 ,
        0.10455938, -0.03992677,  0.13605063,  0.15475224,  0.16

In [72]:
word2vec_model.wv.most_similar(positive=['family', 'town'], negative=['death'], topn=10)

[('they', 0.9957637190818787),
 ('She', 0.9957614541053772),
 ('find', 0.9956477284431458),
 ('for', 0.9956398606300354),
 ('time', 0.9956284761428833),
 ('Christmas', 0.9956049919128418),
 ('has', 0.9956027865409851),
 ('his', 0.9955979585647583),
 ('their', 0.9955976009368896),
 ('the', 0.9955816864967346)]

In [78]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [79]:
# label each of my stories into one of the 4 topic types
# so create data frame w/ plot sum in one column and topic num in next col

In [81]:
# to iterate
class MyDocs(object):
  def __iter__(self):
    for i in range(len(plots_list)):
      yield TaggedDocument(plots_list[i]) ##there are no tags here - need to add
      ## create df and read in df using tags

In [82]:
%%time

if not os.path.exists('models/doc2vec.model'):
    print("start traing doc2vec model...")
    documents = MyDocs()
    doc2vec_model = Doc2Vec(dm=1, dbow_words=1, vector_size=100, window=5, min_count=10, workers=cores)
    doc2vec_model.build_vocab(documents)
    doc2vec_model.train(documents, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
    if not os.path.exists('models'):
        os.makedirs('models')
        doc2vec_model.save('models/doc2vec.model')
    else:
        doc2vec_model.save('models/doc2vec.model')
else:
    doc2vec_model = Doc2Vec.load('models/doc2vec.model')

start traing doc2vec model...


TypeError: ignored